In [1]:
import bs4
import requests
import pandas as pd

# Read in Toronto Data

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df2 = df[0]
df2.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [3]:
type(df2)

pandas.core.frame.DataFrame

# Remove not assigned neighborhoods

In [46]:
for k,v in df2.iterrows():
    if v[2] == 'Not assigned':
        v[2] = v[1]
    else:
        pass
    
df2.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
9,M8A,Not assigned,Not assigned


# Remove not assigned boroughs

In [47]:
df_assigned = df2[df2['Borough'] != 'Not assigned']
df_assigned.head(12)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


# Concatentate the neighbourhoods

In [49]:
df_concat = pd.DataFrame(df_assigned.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)))
df_concat = df_concat.reset_index()
df_concat.head(12)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


# find the new shape

In [25]:
df_concat.shape

(103, 3)

# install geocoder and get the lat and long for each Postcode

In [51]:
!pip install geocoder

     |████████████████████████████████| 102kB 1.1MB/s ta 0:00:01


### geocoder doesn't work so had to download the csv to get the lat and long

In [55]:
import geocoder # import geocoder

# initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#     g = geocoder.google('{}, Toronto, Ontario'.format(df_concat['Postcode']))
#     lat_lng_coords = g.latlng

# df_concat['latitude'] = lat_lng_coords[0]
# df_concat['longitude'] = lat_lng_coords[1]

def geo(df,col):
    g = geocoder.google('{}, Toronto, Ontario'.format(df[col]))
    lat_lng_coords = g.latlng
    return [lat_lng_coords[0],lat_lng_coords[1]]

In [62]:
# for k,v in df_concat.iterrows():
#     a = geo(df_concat,'Postcode')
#     v['latitude'] = a[0]
#     v['longitude'] = a[1]
    
# df_concat

# added lat and long

In [63]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')

In [71]:
df_merged = df_concat.merge(df_geo,left_on=['Postcode'],right_on=['Postal Code'])
df_merged.drop(columns=['Postal Code'], inplace=True)
df_merged.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


# map analysis

In [73]:
!pip install folium
import folium # map rendering library

     |████████████████████████████████| 92kB 1.0MB/s eta 0:00:01


In [79]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Create map of boroughs that have toronto in the name

In [81]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [84]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merged[df_merged['Borough'].str.contains('Toronto')]['Latitude']
                                           ,df_merged[df_merged['Borough'].str.contains('Toronto')]['Longitude']
                                           ,df_merged[df_merged['Borough'].str.contains('Toronto')]['Borough']
                                           ,df_merged[df_merged['Borough'].str.contains('Toronto')]['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##### in Downtown Toronto the best neighborhood to live,from the analysis, richmond. The reason being is close to the financial district and a bunch of venues around the area